# Chatbots

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/extras/use_cases/chatbots/chatbots.ipynb)

## Use case

Chatbots are one of the central LLM use-cases. The core features of chatbots are that they can have long-running conversations and have access to information that users want to know about.

Aside from basic prompting and LLMs, memory and retrieval are the core components of a chatbot. Memory allows a chatbot to remember past interactions, and retrieval provides a chatbot with up-to-date, domain-specific information.

![Image description](/img/chat_use_case.png)

## Overview

The chat model interface is based around messages rather than raw text. Several components are important to consider for chat:

* `chat model`: See [here](/docs/integrations/chat) for a list of chat model integrations and [here](/docs/modules/model_io/models/chat) for documentation on the chat model interface in LangChain. You can use `LLMs` (see [here](/docs/modules/model_io/models/llms)) for chatbots as well, but chat models have a more conversational tone and natively support a message interface.
* `prompt template`: Prompt templates make it easy to assemble prompts that combine default messages, user input, chat history, and (optionally) additional retrieved context.
* `memory`: [See here](/docs/modules/memory/) for in-depth documentation on memory types
* `retriever` (optional): [See here](/docs/modules/data_connection/retrievers) for in-depth documentation on retrieval systems. These are useful if you want to build a chatbot with domain-specific knowledge.

## Quickstart

Here's a quick preview of how we can create chatbot interfaces. First let's install some dependencies and set the required credentials:

In [ ]:
!pip install langchain openai 


With a plain chat model, we can get chat completions by [passing one or more messages](/docs/modules/model_io/models/chat) to the model.

The chat model will respond with a message.

## Memory 

As we mentioned above, the core component of chatbots is the memory system. One of the simplest and most commonly used forms of memory is `ConversationBufferMemory`:
* This memory allows for storing of messages in a `buffer`
* When called in a chain, it returns all of the messages it has stored

LangChain comes with many other types of memory, too. [See here](/docs/modules/memory/) for in-depth documentation on memory types.

For now let's take a quick look at ConversationBufferMemory. We can manually add a few chat messages to the memory like so:

## Conversation 

We can unpack what goes under the hood with `ConversationChain`. 

We can specify our memory, `ConversationSummaryMemory` and we can specify the prompt. 

In [1]:
# Set env var OPENAI_API_KEY or load from a .env file:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
resume = """
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective is to enhance the user experience on our platform by optimizing product search functionality and sorting results based on the best price offers from sellers.
● Leverage historical order data to identify optimal product combinations that maximize gross profit per order. Factors considered include product price, category take rate, unit size, and logistic cost.
● Utilize GPS data from drivers and retailers' addresses to identify and detect abnormal activities from third-party logistic vendors. Uncover and address cases of cheating activities, which have resulted in financial exploitation of the company amounting to $50k, involving three identified vendors.
Katalon — Data Analyst						        November 2021 – November 2022
● Work with the management team on a pricing analysis project to simulate and evaluate an optimal price for the SaaS seat-based model of Katalon Studio, which helps increase 30% of revenue and streamline sales processes with the new business model.
● Collaborate with the Marketing and Product team on a product qualified leads (PQL) project to build a prediction model on users’ usage patterns, and define a list of likely-to-convert customers. The project contributes to an increase in conversion rate from 1.1% to 1.8% compared to the previous year and reduces the sales cycle from 90 days for regular customers to 45 days for PQL customers.
● Initiate Customer Success Health Score project to help customer managers to identify potential churn risks or expansion opportunities by centralizing and leveraging data on product adoption, product experience, customer engagement, and customer sentiment on Salesforce.
● Create star schema data modeling and data mart for building real-time Power BI dashboards to monitor team’s performance and take proactive actions with insightful trends.
KEY ACHIEVEMENT: Co-lead with a data engineer on building a single source of truth data warehouse of the company with Redshift, in charge of building ETL script, data logic, data models, and documents for business users to self-serve the data, and training users on using BI tools. The team’s workload on repeated data requests is reduced by 70%.
Intel Vietnam — Business Data Analyst 			 	January 2021 – November 2021
● Visualize and automate Customs-Accounting assets reconciliation process with Power BI to monitor customs compliance and detect business risks, which helps to reduce reconciliation cycle from 120 days to 7 days.
● Build assets tracking and aging indicator dashboard to optimize 30% of warehouse space utilization and improve operational efficiency.
● Lead assets management business process improvement project, drive collaboration across Accounting, Engineering, Logistics, IT departments to minimize human-dependent errors from over 10 cases to 2 cases per quarter, and align Intel’s business with the Government regulation.
Intel Vietnam — Senior Process Engineer 				  January 2020 – January 2021
● Build automated Python script for collecting and cleaning data to drive and achieve 99% machine availability. 
● Investigate the root cause of quality issues through commonality analysis and model-based problem solving. 
● Create dashboards to monitor product’s yield and auto trigger problematic trends.
Ascenx — Field Service Engineer 				        December 2015 – December 2019 
ANTS — Account Executive 						 August 2014 - November 2015
EDUCATION & SKILLS
Master of Data Science, Ho Chi Minh City University of Technology
2022 - Expected to graduate in 2024 
Chemical Engineering, Ho Chi Minh City University of Technology
Graduated in October 2014
Internship, Arava International Training Center, Israel 
September 2013 - June 2014
Skills: Data analysis, Data modeling, SQL, Python, Pandas, Airflow
Tools: Power BI, PostgreSQL, Redshift, Dbt, Metabase, Dbeaver, Google Analytics, Hubspot.
"""

In [ ]:
"""I want to act as a headhunter assistant to validate and ask screening question to candidate.
At the beginning, you will read the candidate resume. 

Resume: {}

Then you will extract and validate those information with user. 
Step 1: Extract candidate name, phone, email, city, working experiences, top 5 technical skills in below format.

- Name: <name>
- Phone: <phone>
- Email: <email>
- City: <city>
- Top 5 skills: <list of skills>
- Working experience:
    - <company> - <title> - <working date>

Ask them if they are correct or not, if not ask them to input correct information in json format.
I want you as the tone funny, and supportive.
""".format(resume)

In [7]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI


# LLM
llm = ChatOpenAI()

message = """
Step 2: Extract candidate previous companies, roles, working date. Ask them if they are correct or not, if not ask them to input correct information. Or if you can't find the information in the resume, ask them to input.
Step 3: Extract candidate's college and major. Ask them if they are correct or not, if not ask them to input correct information. Or if you can't find the information in the resume, ask them to input.
Step 4: Extract candidate top 5 technical skills. Ask them if they are correct or not, if not ask them to input correct information. Or if you can't find the information in the resume, ask them to input.
Step 5: Ask them their expectation gross salary monthly in USD for better suggestion jobs purpose.
Step 6: Ask candidate their reason for leaving current company
Step 7:  Ask candidate their motivation for finding new opportunity.
"""

# Prompt 
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
"""I want to act as a headhunter assistant to validate and ask screening question to candidate.
At the beginning, you will read the candidate resume. 

Resume: {}

Then you will extract and validate those information with user. 

Step 1: Extract candidate name, phone, email, city, linkedin link, github link, working experiences, top 5 technical skills in below format.

- Name: <name>
- Phone: <phone>
- Email: <email>
- City: <city>
- LinkedIn: <LinkedIn link>
- Github: <Github link>
- Top 5 skills: <list of skills>
- Working experience:
    - <company> - <title> - <working date>

You should ask them if they are correct or not, if not tell them to input correct information.

Step 2: If candidate update their information, you will show the information again for validation. 
If candidate doesn't update anything, just move to step 3.

Step 3: You must ask candidate about his gross annual expected salary for new job.

Step 4: You must ask candidate about his notice period for current job.

Step 5: You must ask candidate about his reason for leaving current or previous job.

Step 6: You must ask candidate about his motivation for applying for this job.

Step 7: You must show all the summarise of the candidate for final validation.

Step 8: Say thank you and encouragement to candidate, and inform them you have applied for the job.

You must ask those above steps one by one.
I want you use the tone funny, encouragement and supportive.
""".format(resume)
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{answer}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

memory.chat_memory.add_ai_message("Hi human, I am Qode - your personal headhunter assistant, I will help you to find\
your next dream job right away. But first, please verify your personal information below:\
<extract information> ")

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
# conversation({"question": "hi"})

In [8]:
response = conversation({"answer": ""})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
At the beginning, you will read the candidate resume. 

Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective is to enh

In [9]:
response = conversation({"answer": "My github link: abc.com"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
At the beginning, you will read the candidate resume. 

Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective is to enh

In [10]:
response = conversation({"answer": "1000$"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
At the beginning, you will read the candidate resume. 

Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective is to enh

In [11]:
response = conversation({"answer": "Immediately"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
At the beginning, you will read the candidate resume. 

Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective is to enh

In [12]:
response = conversation({"answer": "Old technology"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
At the beginning, you will read the candidate resume. 

Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective is to enh

In [13]:
response = conversation({"answer": "Learn new tech"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
At the beginning, you will read the candidate resume. 

Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective is to enh

In [14]:
response = conversation({"answer": "Thank you"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
At the beginning, you will read the candidate resume. 

Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective is to enh


> Finished chain.
You're welcome! Thank you for providing all the necessary information. I will now proceed to apply for the job on your behalf. 

I want to encourage you to stay positive and keep up the great work. Your skills and experience make you a valuable candidate, and I'm confident that you'll find the perfect opportunity soon.

If there's anything else I can assist you with, please don't hesitate to let me know. Good luck with your job search, and have a fantastic day!


In [56]:
for message in memory.buffer:
    if message.__class__.__name__ == 'AIMessage':
        print(f"\033[92m{message.__class__.__name__}: \033[0m" + message.content)
    else:
        print(f"\033[91m{message.__class__.__name__}: \033[0m" + message.content)
    print("-----")  

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to findyour next dream job right away. But first, please verify your personal information below:<extract information> 
-----
HumanMessage: 
-----
AIMessage: Name: NGUYEN PHU QUY
Phone: +84 979 757 305
Email: nguyenphuquy2303@gmail.com
City: [City]
LinkedIn: https://www.linkedin.com/in/nguyenphuquy/
Github: [Github link]
Top 5 skills: Data analysis, Data modeling, SQL, Python, Pandas
Working experience:
1. Kilo MDC - Senior Data Analyst - November 2022 – Present
2. Katalon - Data Analyst - November 2021 – November 2022
3. Intel Vietnam - Business Data Analyst - January 2021 – November 2021
4. Intel Vietnam - Senior Process Engineer - January 2020 – January 2021
5. Ascenx - Field Service Engineer - December 2015 – December 2019

Are these details correct, or would you like to make any changes or updates?
-----
HumanMessage: My github link: abc.com
-----
AIMessage: Great! I've updated your GitHub link to 

We can see the chat history preserved in the prompt using the [LangSmith trace](https://smith.langchain.com/public/dce34c57-21ca-4283-9020-a8e0d78a59de/r).

![Image description](/img/chat_use_case_2.png)

## Chat Retrieval

Now, suppose we want to [chat with documents](https://twitter.com/mayowaoshin/status/1640385062708424708?s=20) or some other source of knowledge.

This is popular use case, combining chat with [document retrieval](/docs/use_cases/question_answering).

It allows us to chat with specific information that the model was not trained on.

In [ ]:
!pip install tiktoken chromadb

Load a blog post.

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

Split and store this in a vector.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

Create our memory, as before, but's let's use `ConversationSummaryMemory`.

In [ ]:
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI()
retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
qa("How do agents use Task decomposition?")

In [ ]:
qa("What are the various ways to implemet memory to support it?")

Again, we can use the [LangSmith trace](https://smith.langchain.com/public/18460363-0c70-4c72-81c7-3b57253bb58c/r) to explore the prompt structure.

### Going deeper 

* Agents, such as the [conversational retrieval agent](/docs/use_cases/question_answering/how_to/conversational_retrieval_agents), can be used for retrieval when necessary while also holding a conversation.
